### RankNet-makedb-laptime

build laptime dataset from raw log files
functions of draw figures moved to PaperFigs

base: 18./laptime_rank_timediff_dataset

create laptime dataset only, no gluonts.

+ add lap2nextpit as target, this is as the same as a stacked lstm model when set prediction_length=1
+ add lap_instint to dynamic features(pitage)
+ add support for historical data, indy500 for different years
+ add track and lap status as target, preparing for StatusModel with deepAR

this notebook focuses on create the training set, while laptime2rank-evaluate will have testset codes specified for rank calculation

+ add new features, caution_laps_instint,lap_instint

Build a time series dataset across all the oval races, including laptime, rank. When this dataset aims to be used in forecasting, covariates of the racing status can not be included, such as track_status and lap_status. However, they can be used in oracle test to tell the upper bound of performance of the predictor.

Change to a new dataset format that following telemetry dataset.

raw:
+  [(eventid, carids: carno -> rowid, datalist)]

datalist := [datalist_entry] in shape of #car_number

datalist_entry := [[laptime, rank, track_status, lap_status]], in shape of #totallaps x #featureCnt (padded by nan)

In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters

import indycar.model.global_variables as gvar
from indycar.model.quicktest_modules import *

register_matplotlib_converters()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [26]:
_savedata = False
outputRoot = 'data/'
os.makedirs(outputRoot, exist_ok=True)

stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0

#inlap status = 
# 0 , no inlap
# 1 , set previous lap
# 2 , set the next lap
_inlap_status = 0
_featureCnt = 9
#
# input data parameters
#
#events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
#events_totalmiles=[256,500,372,268,500,310]
#events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events_info = {
    'Phoenix':(256, 1.022, 250),'Indy500':(500,2.5,200),'Texas':(372,1.5,248),
    'Iowa':(268,0.894,300),'Pocono':(500,2.5,200),'Gateway':(310,1.25,248)
}

# event -> car#, maxlap
_race_info = {}

# the races have 7 years data 
races = ['Indy500', 'Texas','Iowa','Pocono']
years = ['2013','2014','2015','2016','2017','2018','2019']

#_race = 'Indy500'

events = []
for race in races:
    events.extend([f'{race}-{x}' for x in years])

#events.extend(['Phoenix-2018','Texas-2018','Texas-2019','Pocono-2018','Pocono-2019','Iowa-2018','Iowa-2019',
#              'Gateway-2018','Gateway-2019'])
events.extend(['Phoenix-2018','Gateway-2018','Gateway-2019'])

events_id={key:idx for idx, key in enumerate(events)}

#version = f'IndyCar-d{len(events)}'
covergap = 1

#dbid = f'Indy500_{years[0]}_{years[-1]}_v{_featureCnt}_p{_inlap_status}'
dbid = f'IndyCar_d{len(events)}_v{_featureCnt}_p{_inlap_status}'

LAPTIME_DATASET = f'{outputRoot}/laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
STAGE_DATASET = f'{outputRoot}/stagedata-{dbid}.pickle' 
PITCOVERED_DATASET = f'{outputRoot}/pitcoveredlaps-{dbid}-g{covergap}.pickle' 
PITSTOP_DATASET = f'{outputRoot}/pitstop-{dbid}.csv' 

In [27]:
gvar.events = events
gvar.events_id  = events_id
gvar.events_info = events_info
gvar._race_info = _race_info

gvar.dbid = dbid
gvar.LAPTIME_DATASET = LAPTIME_DATASET

In [28]:
for event in events:
    #dataid = f'{event}-{year}'
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event)

    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)-1))
    
    _race_info[event] = (len(carlist), len(laplist)-1, max(laplist))

    #build the carid map
    for car in carlist:
        if car not in global_carids:
            global_carids[car] = cur_carid
            cur_carid += 1


count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 5, 8, 9, 11, 12, 77, 14, 78, 18, 19, 20, 83, 22, 55, 25, 26}
#cars= 19
count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 25, 26, 27, 41, 55, 60, 63, 77, 78, 81, 83, 91, 98}
#cars= 33
Indy500-2013: carno=33, lapnum=200
count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]
cars: {33, 2, 3, 34, 5, 68, 98, 8, 11, 12, 77, 14, 16, 17, 18, 21, 22, 25, 26, 28}
#cars= 20
count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]
cars: {2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 33, 34, 41, 63, 67, 68, 77, 83, 91, 98}
#cars= 33
Indy500-2014: carno=33, lapnum=200
count of completed cars: 20
completed cars: [ 2  1

cars: {2, 5, 7, 8, 9, 10, 11, 12, 41, 15, 21, 26, 27, 28}
#cars= 14
count of completed cars: 14
completed cars: [12  7 28 21 11  9 26  2 10  5 15 27  8 41]
cars: {2, 3, 5, 7, 8, 9, 10, 11, 12, 14, 15, 19, 20, 21, 22, 26, 27, 28, 41, 83, 88, 98}
#cars= 22
Pocono-2016: carno=22, lapnum=200
count of completed cars: 17
completed cars: [12  2 98  1 10  9  3 28 15 14 27 20 26  4 88 83 19]
cars: {1, 2, 3, 4, 98, 9, 10, 12, 14, 15, 19, 20, 83, 88, 26, 27, 28}
#cars= 17
count of completed cars: 17
completed cars: [12  2 98  1 10  9  3 28 15 14 27 20 26  4 88 83 19]
cars: {1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 14, 15, 18, 19, 20, 21, 26, 27, 28, 83, 88, 98}
#cars= 22
Pocono-2017: carno=22, lapnum=200
count of completed cars: 4
completed cars: [27 12  9 18]
cars: {9, 18, 27, 12}
#cars= 4
count of completed cars: 4
completed cars: [27 12  9 18]
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 59, 88, 98}
#cars= 22
Pocono-2018: carno=22, lapnum=200
count of completed cars: 61

In [29]:
laptime_data = get_laptime_dataset(stagedata,inlap_status = _inlap_status)

start event: Indy500-2013
totallaps: Indy500-2013 201 201
event=Indy500-2013, records=(33, 9, 200)
start event: Indy500-2014
totallaps: Indy500-2014 201 201
event=Indy500-2014, records=(33, 9, 200)
start event: Indy500-2015
totallaps: Indy500-2015 201 201
event=Indy500-2015, records=(33, 9, 200)
start event: Indy500-2016
totallaps: Indy500-2016 201 201
event=Indy500-2016, records=(33, 9, 200)
start event: Indy500-2017
totallaps: Indy500-2017 201 201
event=Indy500-2017, records=(33, 9, 200)
start event: Indy500-2018
totallaps: Indy500-2018 201 201
event=Indy500-2018, records=(33, 9, 200)
start event: Indy500-2019
totallaps: Indy500-2019 201 201
event=Indy500-2019, records=(33, 9, 200)
start event: Texas-2013
totallaps: Texas-2013 229 229
event=Texas-2013, records=(24, 9, 228)
start event: Texas-2014
totallaps: Texas-2014 249 249
event=Texas-2014, records=(22, 9, 248)
start event: Texas-2015
totallaps: Texas-2015 249 249
event=Texas-2015, records=(23, 9, 248)
start event: Texas-2016
tota

### get pitstop dataset

In [32]:
def extract_pitstop_features(stagedata):
    """
    input: (alldata, rankdata, acldata, flagdata)
    
    output:
        eventid
        car_number
        pit_id          ; 
    
        lap_number   ; lap_number of this pitstop
        lap_time     ; lap time for this pitstop lap
        total_laps    ; lap distance from the last pit(or start lap)
        total_cautionlaps ; caution laps

        pit_in_caution   ; 1/0
        caution_distance ; lap distance from the start lap of current caution period
    """
    #eventid = 0
    cols=['lap_distance','start_lap','caution_distance',
             'eventid','car_number','stageid',
             'pit_in_caution', 'yellowflag_raised',
             'start_position', 'start_rank', 'end_rank']
    
    pitrec = []
    for event in stagedata.keys():
        
        # get race name
        eventid = events_id[event]
        raceid = event.split('-')[0]
        
        
        alldata, rankdata, acldata, flagdata = stagedata[event]
        carlist = set(acldata['car_number'])
        laplist = set(acldata['completed_laps'])
        totalcars = len(carlist)
        totallaps = len(laplist)

        for carno in carlist:
            
            thiscar = acldata[acldata['car_number']==carno]
            
            #scan the data of this car
            cautionlap_cnt = 0
            lap_cnt = 0
            last_cautionlap_start = -1
            pit_id = 0
            for idx in range(len(thiscar)):
                
                lap_number = thiscar.iloc[idx].completed_laps
                
                if(thiscar.iloc[idx].lap_status == 'P'):
                    if(thiscar.iloc[idx].track_status == 'Y'):
                        pit_oncaution = 1
                        if last_cautionlap_start>0:
                            pit_oncaution_len = lap_number - last_cautionlap_start
                        else:
                            pit_oncaution_len = 0    
                    else:
                        pit_oncaution = 0
                        pit_oncaution_len = 0
                    #a new pit record
                    laptime = thiscar.iloc[idx].time_diff
                    #['eventid', 'carno','pit_id', 'lap_number', 'time_diff',
                    # 'lap_cnt','cautionlap_cnt','pit_oncaution_len','pit_oncaution']
                    pitrec.append([raceid, eventid, carno,pit_id, lap_number, laptime,
                          lap_cnt,cautionlap_cnt,pit_oncaution_len,pit_oncaution])
                    
                    #reset
                    if not pit_oncaution:
                        last_cautionlap_start = -1    
                        
                    cautionlap_cnt = 0
                    lap_cnt = 0
                    
                    pit_id += 1
                    
                else:
                    if(thiscar.iloc[idx].track_status == 'Y'):
                        cautionlap_cnt += 1
                        if (thiscar.iloc[idx-1].track_status != 'Y'):
                            last_cautionlap_start = lap_number
                    else:
                        last_cautionlap_start = -1
                            
                    lap_cnt += 1
                    
        # push this event into stage dataframe
        print('event=%s, records=%d'%(event, len(pitrec)))
    df = pd.DataFrame(pitrec,columns=['raceid', 'eventid', 'carno','pit_id', 'lap_number', 
                     'lap_time',
                     'lap_cnt','cautionlap_cnt','pit_oncaution_len','pit_oncaution'])
        
    return df

In [33]:
pitstop_df = extract_pitstop_features(stagedata)

event=Indy500-2013, records=198
event=Indy500-2014, records=426
event=Indy500-2015, records=624
event=Indy500-2016, records=883
event=Indy500-2017, records=1131
event=Indy500-2018, records=1323
event=Indy500-2019, records=1540
event=Texas-2013, records=1634
event=Texas-2014, records=1756
event=Texas-2015, records=1879
event=Texas-2016, records=2181
event=Texas-2017, records=2333
event=Texas-2018, records=2428
event=Texas-2019, records=2513
event=Iowa-2013, records=2600
event=Iowa-2014, records=2687
event=Iowa-2015, records=2795
event=Iowa-2016, records=2881
event=Iowa-2017, records=2985
event=Iowa-2018, records=3072
event=Iowa-2019, records=3202
event=Pocono-2013, records=3301
event=Pocono-2014, records=3434
event=Pocono-2015, records=3601
event=Pocono-2016, records=3772
event=Pocono-2017, records=3909
event=Pocono-2018, records=4013
event=Pocono-2019, records=4116
event=Phoenix-2018, records=4207
event=Gateway-2018, records=4290
event=Gateway-2019, records=4379


### Get pit-covered-laps

In [47]:
### check the inlap
pitdata = {}
for event in events:    
    alldata, rankdata, acldata, flagdata = stagedata[event]

    totallaps = np.max(rankdata.completed_laps.to_numpy())
    #pitlaps = rankdata[rankdata['lap_status']=='P'][['completed_laps']].to_numpy()
    pitlaps = rankdata[rankdata['lap_status']=='P'].completed_laps.to_numpy()

    pitlaps = set(sorted(pitlaps))

    pitcoveredlaps = []
    for lap in pitlaps:
        gap = range(lap - covergap, min(lap + covergap+1, totallaps))
        #pitcoveredlaps.extend([lap -2,lap-1,lap,lap+1,lap+2])
        pitcoveredlaps.extend(gap)

    pitcoveredlaps = set(sorted(pitcoveredlaps))
    
    print(event, 'total:', totallaps, 'pitlaps:', len(pitlaps), 'pitcoveredlaps:', len(pitcoveredlaps))
    
    #save
    pitdata[event] = [pitlaps, pitcoveredlaps]


Indy500-2013 total: 200 pitlaps: 70 pitcoveredlaps: 119
Indy500-2014 total: 200 pitlaps: 75 pitcoveredlaps: 126
Indy500-2015 total: 200 pitlaps: 48 pitcoveredlaps: 80
Indy500-2016 total: 200 pitlaps: 70 pitcoveredlaps: 120
Indy500-2017 total: 200 pitlaps: 70 pitcoveredlaps: 124
Indy500-2018 total: 200 pitlaps: 76 pitcoveredlaps: 122
Indy500-2019 total: 200 pitlaps: 74 pitcoveredlaps: 102
Texas-2013 total: 228 pitlaps: 55 pitcoveredlaps: 112
Texas-2014 total: 248 pitlaps: 58 pitcoveredlaps: 105
Texas-2015 total: 248 pitlaps: 75 pitcoveredlaps: 136
Texas-2016 total: 248 pitlaps: 85 pitcoveredlaps: 138
Texas-2017 total: 248 pitlaps: 42 pitcoveredlaps: 86
Texas-2018 total: 248 pitlaps: 53 pitcoveredlaps: 112
Texas-2019 total: 248 pitlaps: 49 pitcoveredlaps: 84
Iowa-2013 total: 250 pitlaps: 40 pitcoveredlaps: 71
Iowa-2014 total: 300 pitlaps: 49 pitcoveredlaps: 103
Iowa-2015 total: 300 pitlaps: 50 pitcoveredlaps: 104
Iowa-2016 total: 300 pitlaps: 43 pitcoveredlaps: 81
Iowa-2017 total: 300 pi

In [7]:
_test_event= 'Indy500-2019'
print('pitlaps:', len(pitdata[_test_event][0]), 'coveredlaps:', len(pitdata[_test_event][1]))

pitlaps: 74 coveredlaps: 102


### save data

In [13]:
if _savedata:
    import pickle
    #stintdf.to_csv('laptime-%s.csv'%year)
    savefile = LAPTIME_DATASET
    print(savefile)
    with open(savefile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [global_carids, laptime_data]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)
        
    savefile = STAGE_DATASET 
    print(savefile)
    with open(savefile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [stagedata, _race_info, events, events_id]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)  

    with open(PITCOVERED_DATASET, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = pitdata
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)   

    pitstop_df.to_csv(PITSTOP_DATASET)
        

data//laptime_rank_timediff_pit-oracle-IndyCar_d31_v9_p0.pickle
data//stagedata-IndyCar_d31_v9_p0.pickle


## test

In [38]:
alldata, rankdata, acldata, flagdata = stagedata['Pocono-2019']
np.max(rankdata.completed_laps.values)

128

In [46]:
rankdata[rankdata['completed_laps']==128].head(10)

,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,best_lap,time_behind_leader,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
MyIdx,,,,,,,,,,,,,,,,,,,,,
7353,1,12,4,128,6825.8296,99.7674,P,41.7287,71,0.0000,...,0.0000,0,1,41.7287,Active,Y,6,80,5,1F
7379,2,9,3,128,6831.2984,102.0274,P,42.0286,6C,5.4688,...,5.4688,0,2,42.0286,Active,Y,5,80,4,20
7381,3,22,2,128,6832.9246,100.8849,P,42.0333,54,7.0950,...,1.6262,0,3,42.0333,Active,Y,5,80,3,40
7383,4,19,C,128,6835.2993,101.4892,P,42.0761,46,9.4697,...,2.3747,0,4,42.0761,Active,Y,5,80,D,0
7385,5,2,0,128,6836.0067,96.1433,P,42.4203,5E,10.1771,...,0.7074,0,5,42.4203,Active,Y,5,80,1,1
7388,6,20,E,128,6841.2535,96.8889,P,42.3224,54,15.4239,...,5.2468,0,6,42.3224,Active,Y,5,80,F,0
7390,7,18,A,128,6844.9937,98.1568,P,42.2484,70,19.1641,...,3.7402,0,7,42.2484,Active,Y,5,80,B,0
7393,8,14,12,128,6849.2237,94.5905,P,42.5335,6F,23.3941,...,4.2300,0,8,42.5335,Active,Y,6,80,13,0
7395,9,15,7,128,6851.0356,94.5170,P,42.5903,6B,25.2060,...,1.8119,0,9,42.5903,Active,Y,6,80,8,0


In [10]:
_race_info

{'Indy500-2013': (33, 200, 200),
 'Indy500-2014': (33, 200, 200),
 'Indy500-2015': (33, 200, 200),
 'Indy500-2016': (33, 200, 200),
 'Indy500-2017': (33, 200, 200),
 'Indy500-2018': (33, 200, 200),
 'Indy500-2019': (33, 200, 200),
 'Texas-2013': (24, 228, 228),
 'Texas-2014': (22, 248, 248),
 'Texas-2015': (23, 248, 248),
 'Texas-2016': (22, 248, 248),
 'Texas-2017': (22, 248, 248),
 'Texas-2018': (22, 248, 248),
 'Texas-2019': (22, 248, 248),
 'Iowa-2013': (24, 250, 250),
 'Iowa-2014': (22, 300, 300),
 'Iowa-2015': (24, 300, 300),
 'Iowa-2016': (22, 300, 300),
 'Iowa-2017': (21, 300, 300),
 'Iowa-2018': (22, 300, 300),
 'Iowa-2019': (22, 300, 300),
 'Pocono-2013': (24, 160, 160),
 'Pocono-2014': (22, 200, 200),
 'Pocono-2015': (24, 200, 200),
 'Pocono-2016': (22, 200, 200),
 'Pocono-2017': (22, 200, 200),
 'Pocono-2018': (22, 200, 200),
 'Pocono-2019': (22, 128, 128),
 'Phoenix-2018': (23, 250, 250),
 'Gateway-2018': (21, 248, 248),
 'Gateway-2019': (22, 248, 248)}

In [12]:
_savedata = True

In [34]:
pitstop_df.to_csv(PITSTOP_DATASET)

In [49]:
list(pitdata['Pocono-2019'][1])

[2,
 3,
 4,
 5,
 6,
 9,
 10,
 11,
 12,
 13,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 76,
 77,
 78,
 79,
 80,
 81,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 125,
 126,
 127]

In [50]:
    with open(PITCOVERED_DATASET, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = pitdata
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL) 